In [37]:
!pip install tensorflow

In [42]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN,Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [43]:
#Generating some example sequential data
sentences=['I love coding','I love school','I hate programming','Recurrent Neural Networks are powerful']

In [44]:
#Tokenizing the words
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words=len(tokenizer.word_index) + 1
print(total_words)

12


In [45]:
# Creating input sequences and their corresponding next words
input_sequences = []
for sentence in sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        n_gram_sequence = tokenized_sentence[:i+1]
        input_sequences.append(n_gram_sequence)
input_sequences

[[1, 2],
 [1, 2, 3],
 [1, 2],
 [1, 2, 4],
 [1, 5],
 [1, 5, 6],
 [7, 8],
 [7, 8, 9],
 [7, 8, 9, 10],
 [7, 8, 9, 10, 11]]

In [50]:
# Padding sequences for consistent input size
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
input_sequences

array([[ 0,  0,  0,  1,  2],
       [ 0,  0,  1,  2,  3],
       [ 0,  0,  0,  1,  2],
       [ 0,  0,  1,  2,  4],
       [ 0,  0,  0,  1,  5],
       [ 0,  0,  1,  5,  6],
       [ 0,  0,  0,  7,  8],
       [ 0,  0,  7,  8,  9],
       [ 0,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11]], dtype=int32)

In [54]:
# Creating input and output data
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)
X,y

(array([[ 0,  0,  0,  1],
        [ 0,  0,  1,  2],
        [ 0,  0,  0,  1],
        [ 0,  0,  1,  2],
        [ 0,  0,  0,  1],
        [ 0,  0,  1,  5],
        [ 0,  0,  0,  7],
        [ 0,  0,  7,  8],
        [ 0,  7,  8,  9],
        [ 7,  8,  9, 10]], dtype=int32),
 array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32))

In [49]:
# Building a simple RNN model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=50, input_length=max_sequence_length-1))
model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))

In [53]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [55]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [57]:
# Generating text using the trained model
seed_text = input("Enter the starting word: ")
next_words = int(input("Enter how many words to predict: "))

for _ in range(next_words):
    tokenized_seed = tokenizer.texts_to_sequences([seed_text])[0]
    tokenized_seed = pad_sequences([tokenized_seed], maxlen=max_sequence_length-1, padding='pre')
    predicted_word_index = np.argmax(model.predict(tokenized_seed), axis=-1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    seed_text += " " + predicted_word

print(seed_text)

Enter the starting word: I
Enter how many words to predict: 4
1/1 [==============================] - 0s 22ms/step
I programming programming love i
